In [1]:
# Importando as ferramentas necessárias
import pandas as pd
# Traduz Python para SQL
from sqlalchemy import create_engine
# Segurança
import os
from dotenv import load_dotenv
# Carrega a senha 
load_dotenv()
# Guarda a senha em uma variável
conexao_render = os.environ.get('DATABASE_URL')
# Criando a conexão com o banco de dados
engine = create_engine(conexao_render)
# Testando o SQL
query = "SELECT * FROM clientes LIMIT 5"
print("Buscando os dados no Render")
# Leva a query e traz a resposta, colocando-a em um DataFrame
df_amostra = pd.read_sql(query, engine)
df_amostra



Buscando os dados no Render


,id_cliente,renda_mensal,score_interno,estado_civil
0,1,5245.071230,727,Solteiro
1,2,4292.603548,604,Casado
2,3,5471.532807,616,Solteiro
3,4,6784.544785,934,Divorciado
4,5,4148.769938,308,Divorciado


In [3]:
# Verificando as linhas e colunas da tabela faturas
query_faturas = "SELECT * FROM faturas LIMIT 5"
df_amostra_faturas = pd.read_sql(query_faturas, engine)
df_amostra_faturas

,id_cliente,mes_referencia,valor_fatura,pago_em_dia
0,1,1,4954.620957,1
1,1,2,4974.313479,1
2,1,3,211.124230,1
3,1,4,1840.555543,0
4,1,5,531.341796,1


In [4]:
# Criandouma query para responder o seguinte questionamento: Quais são os clientes com o maior número de faturas atrasadas e qual o Score deles?
query_faturas_atrasadas = """
SELECT
    c.id_cliente,
    c.score_interno,
    c.renda_mensal,
    COUNT(f.id_cliente) as total_faturas,
    SUM(CASE WHEN f.pago_em_dia = 0 THEN 1 ELSE 0 END) as quantidade_faturas_atrasadas
FROM clientes as c
join faturas as f
    on c.id_cliente = f.id_cliente
GROUP BY
    c.id_cliente, c.score_interno, c.renda_mensal
ORDER BY
    quantidade_faturas_atrasadas DESC
LIMIT 10;
""" 
print("Analisando a quantidade de faturas atrasadas por clientes e cruzando os dados no Render")
df_faturas_atrasadas = pd.read_sql(query_faturas_atrasadas, engine)
df_faturas_atrasadas

Analisando a quantidade de faturas atrasadas por clientes e cruzando os dados no Render


,id_cliente,score_interno,renda_mensal,total_faturas,quantidade_faturas_atrasadas
0,268,384,2354.787933,6,5
1,75,416,1621.000000,6,4
2,6,372,4148.794565,6,4
3,892,375,5260.911047,6,4
4,584,379,7906.039287,6,4
5,238,401,4779.681472,6,4
6,315,389,6458.218211,6,4
7,793,387,4907.368256,6,4
8,843,313,2815.916968,6,4
9,533,409,4400.880302,6,4


## O valor da fatura influencia se ela é paga ou não?

In [ ]:
# Analisando se o valor das faturas influencia no aso
query_valor_faturas = """
SELECT
    CASE WHEN pago_em_dia = 0 THEN 'Atrasada' ELSE 'Paga em dia' END as status_fatura,
    AVG(valor_fatura) as valor_medio_fatura,
    COUNT(*) as quantidade_faturas
FROM faturas
GROUP BY 
    pago_em_dia;
"""
print("Analisando o valor médio das faturas")
df_valor_medio = pd.read_sql(query_valor_faturas, engine)
df_valor_medio

Analisando o valor médio das faturas


DatabaseError: Execution failed on sql '
SELECT
    id_cliente,
    CASE WHEN pago_em_dia = 0 THEN 'Atrasada' ELSE 'Paga em dia' END as status_fatura,
    AVG(valor_fatura) as valor_medio_fatura,
    COUNT(*) as quantidade_faturas
FROM faturas
GROUP BY 
    pago_em_dia;
': (psycopg2.errors.GroupingError) column "faturas.id_cliente" must appear in the GROUP BY clause or be used in an aggregate function
LINE 3:     id_cliente,
            ^

[SQL: 
SELECT
    id_cliente,
    CASE WHEN pago_em_dia = 0 THEN 'Atrasada' ELSE 'Paga em dia' END as status_fatura,
    AVG(valor_fatura) as valor_medio_fatura,
    COUNT(*) as quantidade_faturas
FROM faturas
GROUP BY 
    pago_em_dia;
]
(Background on this error at: https://sqlalche.me/e/20/f405)